## Install invasive species mapping repo

In [1]:
%pip install watermark geemap geeml -q
!git clone https://github.com/Geethen/Invasive_Species_Mapping.git
import sys
sys.path.insert(0,'/content/Invasive_Species_Mapping/code')

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip
fatal: destination path 'Invasive_Species_Mapping' already exists and is not an empty directory.


In [2]:
%load_ext watermark

In [13]:
import ee
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()
import geemap
from tqdm.auto import tqdm

# Add module to environment varibles
import sys
MODULE_FULL_PATH = r'C:\Users\coach\myfiles\postdoc\code\Invasive_Species_Mapping\code'
sys.path.insert(1, MODULE_FULL_PATH)

# Load python modules with preprocessing functions.
from geeml.utils import eeprint

In [4]:
%watermark -v -m --iversions

Python implementation: CPython
Python version       : 3.9.13
IPython version      : 8.4.0

Compiler    : MSC v.1929 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 165 Stepping 2, GenuineIntel
CPU cores   : 12
Architecture: 64bit

ee    : 0.2
geemap: 0.17.1
sys   : 3.9.13 | packaged by conda-forge | (main, May 27 2022, 16:50:36) [MSC v.1929 64 bit (AMD64)]



## Import DynamicWorld validation

In [5]:
dynamicWorldValidation = ee.ImageCollection('projects/nina/GIS_synergy/Extent/DW_global_validation_tiles')\
  .select([1], ['label']).map(lambda img: img.updateMask(img.neq(0)).subtract(1).copyProperties(img))
validation = dynamicWorldValidation#.limit(20)

In [12]:
DW =ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").filterDate('2020-01-01', '2021-01-01').select(pClassNames)
# class names
pClassNames = ee.List(['water', 'trees', 'grass', 'flooded_vegetation',
'crops', 'shrub_and_scrub', 'built', 'bare', 'snow_and_ice'])



In [20]:
# sanity check with single image
imgN = dynamicWorldValidation.size()
imgList = dynamicWorldValidation.toList(1)

# Define region- used in export. Even though this is a global extent, the region with data is only exported
globalBounds = ee.Geometry.Polygon([-180, 90, 0, 90, 180, 90, 180, -90, 10, -90, -180, -90], None, False)

# Loop through each image in the validation dataset, add it to the median 2020 prob image and export to imagecollection
for i in tqdm(range(0, 1)):
    img = ee.Image(imgList.get(i))
    out = DW.filterBounds(img.geometry()).median().addBands(img)
    export_params1 = {
    'image': out,
    'description': f'export_{i}',
    'assetId': f'projects/ee-geethensingh/assets/UQ/DW/img_{i}',
    'scale': 10,
    'region': ee.Algorithms.ProjectionTransform(out.geometry(), 'EPSG:4326'),
    'crs': 'EPSG:4326',
    'maxPixels': 1e13}
    # Export the data to the Earth Engine asset
    task = ee.batch.Export.image.toAsset(**export_params1)
    task.start()

  0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
imgN = dynamicWorldValidation.size()
imgList = dynamicWorldValidation.toList(dynamicWorldValidation.size())


# Loop through each image in the validation dataset, add it to the median 2020 prob image and export to imagecollection
for i in tqdm(range(0, imgN.getInfo())):
    img = ee.Image(imgList.get(i))
    out = dwMedian.addBands(img)
    export_params1 = {
    'image': out,
    'description': f'export_{i}',
    'assetId': f'projects/ee-geethensingh/assets/UQ/DW/img_{i}',
    'scale': 10,
    'region': globalBounds,
    'crs': 'EPSG:4326',
    'maxPixels': 1e13}
    # Export the data to the Earth Engine asset
    task = ee.batch.Export.image.toAsset(**export_params1)
    task.start()

  0%|          | 0/409 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Extract labels and covariates from composite

In [10]:
# https://code.earthengine.google.com/d84c5762ba910522004534636f59f0b8
# name of the label property
label = 'label'

# Define region- used to determine extent for sampling
globalBounds = ee.Geometry.Polygon([-180, 90, 0, 90, 180, 90, 180, -90, 10, -90, -180, -90], None, False)

#  Get a stratified sample of each of 9 classes using the label band.
data = ee.FeatureCollection(validation.limit(2).map(lambda img: dwFiltered.addBands(ee.Image(img))
.stratifiedSample(numPoints=10, classBand='label', region = globalBounds,scale=10, seed=42, geometries= True))).flatten()


In [11]:
# data = ee.FeatureCollection("projects/ee-geethensingh/assets/UQ/dwPoints")
data.size()

In [27]:
# https://code.earthengine.google.com/0fca21154cca817d73d9e8fe845c6cf1
# export training data 
# https://code.earthengine.google.com/75c50e8f70ffb9114b1d1346183b38bd
# https://code.earthengine.google.com/0e5719b7dd9972b8cdcdb5a108b3998d
# https://code.earthengine.google.com/81a7640cb252db44e4c6cadfb4d874e0: compute scores in image space


In [8]:
bandnames = pClassNames
# number of classes
NCLASS = data.aggregate_array('label').distinct().length().getInfo()

## Uncertainty Quantification

In [11]:
# Split stratified sample into calibration and test data.
# calibration data used to calibrate conformal predictor and test data
# used to asess coverage

SPLIT = 0.5
#  Add a random column (by default named 'random')
data = data.randomColumn(seed = 42)
# Split featurecollection into calibration and test data
calibration = data.filter(ee.Filter.lt('random', SPLIT))
test = data.filter(ee.Filter.gte('random', SPLIT))

In [12]:
calibration.size()

In [13]:
test.size()

In [14]:
%load_ext autoreload

In [15]:
%autoreload 3

from conformalClassifier import conformalClassifier
# Initialise Conformal Classifier
cc = conformalClassifier(dwFiltered, calibration, test, label, 0.1)
# Calibrate conformal classifier and compute set Masks and set Lengths per pixel
conformalMasks = cc.quantifyImageUncertainty()
# Evaluate conformal classifier on test set -> returns average set size and empirical coverage for test set 
# evalStats = cc.evaluateUncertainty()

In [17]:
eeprint(conformalMasks)

EEException: Can't transform (-116335.48916346629,-383280.8873345152)